### Load input data

In [ ]:
import pandas as pd

df_h = pd.read_csv(f"data/data_hourly.csv.bz2", index_col=0, parse_dates=True)
df_d = pd.read_csv(f"data/data_daily.csv.bz2", index_col=0, parse_dates=True)

## Breakdown of node traffic by connection type

In [ ]:
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from matplotlib.ticker import EngFormatter

df = df_d.copy()


df_pivot = df.pivot_table(
    index=df.index, columns="peer_conn_type", values="net_size", aggfunc="sum"
)

df_pivot = df_pivot[["feeler", "block-relay-only", "outbound-full-relay", "inbound"]]


df_pivot = df_pivot.fillna(0)
df_pivot = df_pivot.sort_index()
x = df_pivot.index
y = df_pivot.T.values
labels = df_pivot.columns

fig, ax = plt.subplots()
fig.suptitle("Total traffic by connection type")
ax.set_title("Breakdown of daily node P2P TCP/IP traffic by connection type")
ax.stackplot(x, y, labels=labels)
formatter = EngFormatter(unit="B")
ax.set_yscale("log")
ax.yaxis.set_major_formatter(formatter)
ax.xaxis.set_major_locator(mdates.DayLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %d"))
ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
ax.legend(ncol=4, title="")

plt.tight_layout()
plt.show()
fig.savefig(
    "total_traffic_by_connection_type.png",
    dpi=300,
    bbox_inches="tight",
    facecolor="white",
)

## Plot daily contribution per connection type

In [ ]:
DEFAULT_NUM_CONNECTIONS = {
    "feeler": 1,
    "block-relay-only": 2,
    "outbound-full-relay": 8,
    "inbound": 125 - (8 + 2),
}

df_per_conn = pd.DataFrame()
fig, ax = plt.subplots()
fig.suptitle("Per-connection traffic by connection type")

for col, num_conns in DEFAULT_NUM_CONNECTIONS.items():
    df_per_conn[col] = df_pivot[col] / num_conns
    ax.plot(df_per_conn.index, df_per_conn[col], label=col)


ax.set_title("Daily TCP/IP traffic for a single connection by connection type")
ax.set_yscale("log")
formatter = EngFormatter(unit="B")
ax.yaxis.set_major_formatter(formatter)
ax.xaxis.set_major_locator(mdates.DayLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %d"))
ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
ax.legend(ncol=4, title="")

plt.tight_layout()
plt.show()
fig.savefig(
    "per_connection_traffic_by_connection_type.png",
    dpi=300,
    bbox_inches="tight",
    facecolor="white",
)